In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import yfinance as yahooFinance
from backtesting.test import SMA

from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn import svm
from sklearn.metrics import classification_report


ModuleNotFoundError: No module named 'backtesting'

In [ ]:
GetFacebookInformation = yahooFinance.Ticker("META")
df=pd.DataFrame(data=GetFacebookInformation.history(period="max"))
df

In [ ]:
# Calculate the daily returns using the closing prices and the pct_change function
df["Actual_Returns"] = df["Close"].pct_change()

# Display sample data
df.head()

In [ ]:
# Drop all NaN values from the DataFrame
df = df.dropna()

# Review the DataFrame
display(df.head())
display(df.tail())

# Baseline Trading Algorithm

In [ ]:
signals_df = df

In [ ]:
# Set the short window and long window
short_window = 4
long_window = 100

# Generate the fast and slow simple moving averages (4 and 100 days, respectively)
signals_df['SMA_Fast'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['Close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Initialize the new Signal column
signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual_Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual_Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
signals_df['Signal'].value_counts()

In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy_Returns'] = signals_df['Actual_Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy_Returns']).cumprod().plot()

In [ ]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

# Review the DataFrame
X.head()

In [ ]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

In [ ]:
train_duration = 6  # months
step_size = 4  # month

# Initialize lists to store training and testing dataframes
all_X_train = []
all_y_train = []
all_X_test = []
all_y_test = []

# Initialize the starting point for the training period
current_start = X.index.min()

# Loop until the end of the dataset
while current_start + pd.DateOffset(days=train_duration) <= X.index.max():
    # Define the training end period
    training_end = current_start + pd.DateOffset(days=train_duration)

    # Generate the training data for the current window
    X_train = X.loc[current_start:training_end]
    y_train = y.loc[current_start:training_end]
    all_X_train.append(X_train)
    all_y_train.append(y_train)

    # Define the testing period as the next time step after the training end
    testing_end = training_end + pd.DateOffset(days=step_size)
    if testing_end > X.index.max():
        testing_end = X.index.max()

    # Generate the testing data for the current window
    X_test = X.loc[training_end:testing_end]
    y_test = y.loc[training_end:testing_end]
    all_X_test.append(X_test)
    all_y_test.append(y_test)

    # Update the start for the next iteration
    current_start = testing_end

# Concatenate all training and testing dataframes
final_X_train = pd.concat(all_X_train)
y_train = pd.concat(all_y_train)
final_X_test = pd.concat(all_X_test)
y_test = pd.concat(all_y_test)

In [ ]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X_train data
X_scaler = scaler.fit(final_X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(final_X_train)
X_test_scaled = X_scaler.transform(final_X_test)

In [ ]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index = final_X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual_Returns'] = signals_df['Actual_Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy_Returns'] = predictions_df['Actual_Returns'] * predictions_df['Predicted']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['Actual_Returns', 'Strategy_Returns']]).cumprod().plot(figsize = (15,5))

# LogisticRegression

In [ ]:
# Import a new classifier from SKLearn
from sklearn.linear_model import LogisticRegression

In [ ]:
# Initiate the model instance
logistic_regression_model = LogisticRegression()

In [ ]:
# Fit the model using the training data
model = logistic_regression_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
pred = logistic_regression_model.predict(X_test_scaled)

# Review the model's predicted values
pred

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
signal_predictions = classification_report(y_test, pred)

# Print the classification report
print(signal_predictions)

In [ ]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
second_predictions_df = pd.DataFrame(index=final_X_test.index)

# Add the SVM model predictions to the DataFrame
second_predictions_df['Predicted'] = pred

# Add the actual returns to the DataFrame
second_predictions_df["Actual_Returns"] = signals_df["Actual_Returns"]

# Add the strategy returns to the DataFrame
second_predictions_df['Strategy_Returns'] = second_predictions_df["Actual_Returns"] * pred

# Review the DataFrame
second_predictions_df

In [ ]:
print("Model Predictions (pred):")
print(pred)

print("Actual Returns:")
print(second_predictions_df["Actual_Returns"])

print("Strategy Returns:")
print(second_predictions_df["Strategy_Returns"])


In [ ]:
# Plot the actual returns versus the strategy returns
(1 + second_predictions_df[["Actual_Returns", "Strategy_Returns"]]).cumprod().plot(figsize = (15,5))

# Multi-layer Perceptron (MLP) Algorithm

In [ ]:
# Import a new classifier from SKLearn

# multi-layer perceptron (MLP) algorithm
from sklearn.neural_network import MLPClassifier

# Initiate the model instance
clf = MLPClassifier(solver='sgd',
                    alpha=1e-5,
#                    max_iter=1000,
                    hidden_layer_sizes=(8, 4),
                    random_state=42
                    )

In [ ]:
# Fit the model using the training data
MLP_model = clf.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
MLP_pred = MLP_model.predict(X_test_scaled)

# Review the model's predicted values
MLP_pred

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
MLP_testing_report = classification_report(y_test, MLP_pred)

# Print the classification report
print(MLP_testing_report)

In [ ]:
# Create a predictions DataFrame
MLP_predictions_df = pd.DataFrame(index = final_X_test.index)

# Add the model predictions to the DataFrame
MLP_predictions_df['Predicted'] = MLP_pred

# Add the actual returns to the DataFrame
MLP_predictions_df['Actual_Returns'] = signals_df['Actual_Returns']

# Add the strategy returns to the DataFrame
MLP_predictions_df['Strategy_Returns'] = MLP_predictions_df['Actual_Returns'] * MLP_predictions_df['Predicted']

# Review the DataFrame
display(MLP_predictions_df.head())
display(MLP_predictions_df.tail())

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + MLP_predictions_df[['Actual_Returns', 'Strategy_Returns']]).cumprod().plot(figsize = (15,5))